In [1]:
import os
import parmed as pmd
from Bio.PDB import *
import pdb4amber
import numpy as np
import subprocess
basepath = os.getcwd()

In [2]:
!cp ../../US-final/noSASA/src . -r
!cp ../../US-final/noSASA/batch_window.sh .

In [2]:
def reload_struct(path):
    parser = PDBParser()
    return parser.get_structure('halfpdb', path)

def move_chain(d, chain):
    for res in chain:
        for atom in res:
            newcoord = atom.get_coord()
            newcoord[2] = newcoord[2] + d
            atom.set_coord(newcoord)
    return

def save_struct(name):
    if not name.endswith('.pdb'):
        name += '.pdb'
    io = PDBIO()
    io.set_structure(structure)
    io.save(name)
    return

def generate_rst7(tleap_string, dist):
    with open(f'{dist}/resource/tleap_script.in','w') as f:
        f.write(tleap_string.format(dist))
    subprocess.run(['/cm/shared/package/amber/20_v2/bin/tleap', '-f', f'{dist}/resource/tleap_script.in'])
    return
                   
def write_plumed_file(plumed_string, dist):
    with open(f'{dist}/prod/umb-{dist}.dat', 'w') as f:
        f.write(plumed_string.format(dist))
    return

def create_folders(dist):
    os.mkdir(str(dist))
    os.mkdir((str(dist) + '/prod'))
    os.mkdir((str(dist) + '/heat'))
    os.mkdir((str(dist) + '/EM'))
    os.mkdir((str(dist) + '/resource'))  
    return

def create_EM_param(em_string, dist):
    with open(f'{dist}/EM/EM.in','w') as f:
        f.write(em_string)
    return

def create_heat_param(heat_string, dist, temp=300.0):
    with open(f'{dist}/heat/heat.in','w') as f:
        f.write(heat_string.format(temp))
    return

def create_prod_param(prod_string, dist, temp=300.0):
    with open(f'{dist}/prod/prod.in','w') as f:
        f.write(prod_string.format(temp, dist))
    return

def convert_pdb(pdb_file, outfile):
    pdb4amber.run(arg_pdbin=pdb_file, arg_pdbout=outfile)
    return

def write_plumed_file(plumed_string, temp):
    with open(f'{temp}/prod/metad-{temp}.dat', 'w') as f:
        f.write(plumed_string.format(temp))
    return

In [3]:
EM_STRING = """IW Minimization
 &cntrl
   imin=1,       ! Run minimization
   ntx=1,        ! Read only coordinates, no velocities
   irest=0,      ! Do not restart
   maxcyc=10000, ! Max 10000 cycles
   ncyc=2000,    ! Steepest descent first 2000 cycles
   ntpr=1000,    ! Update mdout every 1000 cycles
   ntwx=0,       ! No trajectories
   igb=5,        ! GB model
   gbsa=3,       ! SASA is calculated using GPU
   cut=1000,     ! Necesarry for GPU
   saltcon=0.1,  ! salt conc. in M
 /
 """

HEAT_STRING = """IW Heating
 &cntrl
  imin=0,         ! This is an actual MD run
  ntx=1,          ! Read only coordinates, no velocities.
  irest=0,        ! Do not restart.
  nstlim=10000,   ! Number of MD steps (10000*0.002 = 20 ps)
  dt=0.002,       ! Time length of each MD step in ps.
  ntf=2,          ! Do not calculate force for SHAKE constrained bonds
  ntc=2,          ! Allow SHAKE to contrain all bonds
  tempi=0.0,      ! Initial temperature
  temp0={0},    ! Target temperature
  ntpr=100,       ! Print mdout info every 100 cycles.
  ntwx=1000,      ! Write trajectory info every 10000 steps.
  ntb=0           ! No periodic boundary condition
  ntt=3,          ! Langeving thermostat
  gamma_ln=2.0,   ! Colission frequency for langevin
  nmropt=1,       ! NMR restraints
  ig=-1,          ! Randomize the seed
  igb=5,
  gbsa=3,
  saltcon=0.1,
  cut=1000
 /  
&wt type='TEMP0', istep1=0, istep2=9000, value1=0.0, value2={0} /  
&wt type='TEMP0', istep1=9001, istep2=10000, value1={0}, value2={0} /  
&wt type='END' /
"""  

PROD_STRING = """Window run
 &cntrl
  imin=0,  
  ntx=5,  
  irest=1,  
  nstlim=50000000,  ! 25 ns
  dt=0.002,  
  ntf=2,  
  ntc=2,  
  temp0={0},  
  ntpr=2000,  ! Save info every 4 ps
  ntwx=2000,  ! Save traj every 4 ps
  ntb=0,
  ntp=0,    
  ntt=3,  
  barostat=1,  
  gamma_ln=1.0,  
  ig=-1,
  igb=5,
  gbsa=3,
  saltcon=0.1,
  cut=1000,
  plumed=1,
  plumedfile='{0}/prod/metad-{0}.dat'
 /
 """

PLUMED_STRING = """INCLUDE FILE=src/plumed.dat
metad: METAD ... 
    ARG=dist 
    SIGMA=0.1 
    HEIGHT=0.5 
    PACE=250 
    FILE={0}/HILLS
    GRID_MIN=-0.5 
    GRID_MAX=9.5 
    GRID_BIN=500 
    INTERVAL=0,7.0 
    CALC_RCT
...

uwall: UPPER_WALLS ...
    ARG=dist 
    AT=7.0 
    KAPPA=1000
...

PRINT ARG=* STRIDE=100 FILE={0}/COLVAR
"""

TLEAP_STRING = """source leaprc.protein.ff19SB
struct = loadPDB {0}/resource/halfpdb_cleaned.pdb
setbox struct vdw
saveamberparm struct {0}/resource/top.parm7 {0}/resource/start.rst7
quit"""


In [3]:
temps = [280, 300, 320]
dist = 1.5

In [51]:
# Loop over temperatures
for temp in temps:
    
    # Create param files
    create_folders(temp)
    create_EM_param(EM_STRING, temp)
    create_heat_param(HEAT_STRING, temp, temp)
    create_prod_param(PROD_STRING, temp, temp)
    write_plumed_file(PLUMED_STRING, temp)
    
    # Generate starting config
    structure = reload_struct('src/halfpdb_nosep.pdb')
    chainA = structure[0]['A']
    move_chain(dist * 10, chainA)
    save_struct(f'{temp}/resource/{temp}.pdb')

    # Convert config 
    convert_pdb(f'{temp}/resource/{temp}.pdb', f'{temp}/resource/halfpdb_cleaned.pdb')
    generate_rst7(TLEAP_STRING, temp)
        
    # Go back to original dir
    os.chdir(basepath)

-I: Adding /cm/shared/package/amber/20_v2/dat/leap/prep to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/lib to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/parm to search path.
-I: Adding /cm/shared/package/amber/20_v2/dat/leap/cmd to search path.
-f: Source 280/resource/tleap_script.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./280/resource/tleap_script.in
----- Source: /cm/shared/package/amber/20_v2/dat/leap/cmd/leaprc.protein.ff19SB
----- Source of /cm/shared/package/amber/20_v2/dat/leap/cmd/leaprc.protein.ff19SB done
Log file: ./leap.log
Loading parameters: /cm/shared/package/amber/20_v2/dat/leap/parm/parm19.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA + ff19SB
Loading parameters: /cm/shared/package/amber/20_v2/dat/leap/parm/frcmod.ff19SB
Reading force field modification type file (frcmod)
Reading title:
ff19SB AA-specific backbone CMAPs for protein 07/25/2019
Loading library: /cm/shared/package/amber

In [4]:
for temp in temps:
    command = f'sbatch -J meta{temp} batch_window-noscratch.sh -e -s {temp} -c {temp}/resource/start.rst7 -t {temp}/resource/top.parm7'
    print(command)
    os.system(command)

os.chdir(basepath)

sbatch -J meta280 batch_window-noscratch.sh -e -s 280 -c 280/resource/start.rst7 -t 280/resource/top.parm7
Submitted batch job 128077
sbatch -J meta300 batch_window-noscratch.sh -e -s 300 -c 300/resource/start.rst7 -t 300/resource/top.parm7
Submitted batch job 128078
sbatch -J meta320 batch_window-noscratch.sh -e -s 320 -c 320/resource/start.rst7 -t 320/resource/top.parm7
Submitted batch job 128079
